In [1]:
import os
import pandas as pd 
import math
import numpy as np 
from statsmodels.sandbox.stats.multicomp import multipletests

In [2]:
# Initial PATH and Folder Setup
PATH='/Users/houy2/Documents/projects/AD/AD_genetics/results/update_v9_Jul072023/MR_results'
LD="LD02" #LD clumping threshold: square(R) = 0.2
DIR_mr=f'AD_MR_{LD}_Jul172023'
DIR_mr_raw=f'AD_MR_{LD}_Jul172023/raw_xQTL'
DIR_mr_processed=f'AD_MR_{LD}_Jul172023/processed'
DIR_alzMR=f"AD_MR_{LD}_Jul172023/alzMR_DT" #the folder include alzMR score.
file_type=f"AD#{LD}"
update_date="Jul17"

In [3]:
os.chdir(PATH)
listname = os.listdir("./%s" % DIR_mr_raw)
print(f"The number of MR results across xQTL* AD_GWAS pairs:#{len(listname)}")

The number of MR results across xQTL* AD_GWAS pairs:#3130


In [4]:
#Step1, we combined all MR results by all methods. 
##Combine all MR results using IVW when the number of IVs =1 OR IV = 1.
df_snp2 = pd.DataFrame()
for f in listname:
    if "snp" in f:
        df=pd.read_csv("./%s/%s" % (DIR_mr_raw,f), sep="\t")
        df['Outcome']= f.split(".")[0].split('#')[1]
        xQTL=f.split(".")[0].split('#')[2]
        df['xQTL_type'] =xQTL
        df_snp2 = pd.concat([df, df_snp2])
df_snp2=df_snp2[['Outcome','exposure','xQTL_type','method','nsnp','b','lo_ci','up_ci','se','pval','FDR']]
df_snp2.rename(columns={'exposure':'Exposure','nsnp':'SNPs','b':'Estimate','se':'StdError','lo_ci':'CILower','up_ci':'CIUpper','pval':'Pvalue'},inplace=True)
df_snp2.to_csv("./%s/%s_snp2_%s.tsv" % (DIR_mr_processed,file_type,update_date),index=False,sep='\t')
df_snp2.head()

,Outcome,Exposure,xQTL_type,method,SNPs,Estimate,CILower,CIUpper,StdError,Pvalue,FDR
0,AD_NMAFallNoUKB23_b37_2021N,ZNF408,Flead_map_eQTL_brain_mayometa_fdr,Inverse variance weighted,2,-0.110594,-0.263294,0.042106,0.077908,0.155740,0.985086
1,AD_NMAFallNoUKB23_b37_2021N,ALMS1P,Flead_map_eQTL_brain_mayometa_fdr,Inverse variance weighted,2,-0.018612,-0.181386,0.144163,0.083048,0.822674,0.985086
2,AD_NMAFallNoUKB23_b37_2021N,UBE2R2,Flead_map_eQTL_brain_mayometa_fdr,Inverse variance weighted,2,0.074615,-0.132360,0.281589,0.105599,0.479825,0.985086
3,AD_NMAFallNoUKB23_b37_2021N,NACC1,Flead_map_eQTL_brain_mayometa_fdr,Inverse variance weighted,2,0.070275,-0.050416,0.190965,0.061577,0.253767,0.985086
4,AD_NMAFallNoUKB23_b37_2021N,DCAF8,Flead_map_eQTL_brain_mayometa_fdr,Inverse variance weighted,2,0.055664,-0.056125,0.167453,0.057035,0.329084,0.985086


In [5]:
#Combine all MR results using IVW when the number of IVs >=3.
df_IVW = pd.DataFrame()
for f in listname:
    if "IVW" in f:
        df=pd.read_csv("./%s/%s" % (DIR_mr_raw,f), sep="\t")
        df['Outcome']= f.split(".")[0].split('#')[1]
        xQTL=f.split(".")[0].split('#')[2]
        df['xQTL_type'] =xQTL
        df_IVW = pd.concat([df, df_IVW])
df_IVW.to_csv("./%s/%s_IVW3_%s.tsv" % (DIR_mr_processed,file_type,update_date),index=False,sep='\t')
df_IVW.head()

,Exposure,Outcome,SNPs,method,Estimate,StdError,CILower,CIUpper,Pvalue,Heter.Stat[1],Heter.p,method_type,FDR,xQTL_type
0,GRPEL1,AD_Jansen_2019N,17,IVW,-0.006078,0.003373,-0.012690,0.000534,0.071597,20.795037,0.186524,IVW,0.448229,Flead_map_eQTL_brain_mayometa_fdr
1,RP11-529E10.6,AD_Jansen_2019N,9,IVW,0.008785,0.005552,-0.002098,0.019667,0.113607,9.456025,0.305297,IVW,0.583433,Flead_map_eQTL_brain_mayometa_fdr
2,TTLL5,AD_Jansen_2019N,18,IVW,0.002433,0.002713,-0.002885,0.007751,0.369937,19.598959,0.295254,IVW,0.848809,Flead_map_eQTL_brain_mayometa_fdr
3,TDO2,AD_Jansen_2019N,15,IVW,-0.002425,0.002702,-0.007721,0.002872,0.369600,11.728905,0.628064,IVW,0.848809,Flead_map_eQTL_brain_mayometa_fdr
4,PHF20L1,AD_Jansen_2019N,6,IVW,0.000831,0.007901,-0.014654,0.016316,0.916232,1.850102,0.869473,IVW,0.995892,Flead_map_eQTL_brain_mayometa_fdr


In [6]:
#Combined results from the Wald ratio and IVW.
df_Wald_IVW = pd.concat([df_snp2, df_IVW])
df_Wald_IVW =df_Wald_IVW.dropna(subset=["Exposure"])
df_Wald_IVW.to_csv("./%s/%s_IVW_%s.tsv" % (DIR_mr_processed,file_type,update_date),index=False,sep='\t')
print(df_Wald_IVW.shape)

(576902, 14)


In [7]:
#Combined results from the Maxlik, Weighted median and MRpresso.
df_Maxlik = pd.DataFrame()
df_WM = pd.DataFrame()
df_MP = pd.DataFrame()
for f in listname:
    #print(f)
    if "Maxlik#" in f:
        df=pd.read_csv("./%s/%s" % (DIR_mr_raw,f), sep="\t")
        df['Outcome']= f.split(".")[0].split('#')[1]
        xQTL=f.split(".")[0].split('#')[2]
        df['xQTL_type'] =xQTL
        df = df.dropna(subset=['Exposure','FDR']).copy()
        df_Maxlik = pd.concat([df, df_Maxlik])
    elif "WM#" in f:
        df=pd.read_csv("./%s/%s" % (DIR_mr_raw,f), sep="\t")
        df['Outcome']= f.split(".")[0].split('#')[1]
        xQTL=f.split(".")[0].split('#')[2]
        df['xQTL_type'] =xQTL
        df = df.dropna(subset=['Exposure','FDR']).copy()
        df_WM = pd.concat([df, df_WM])
    elif "MRpresso#" in f:
        df=pd.read_csv("./%s/%s" % (DIR_mr_raw,f), sep="\t")
        df = df.drop(['outcome','T-stat','xqtl'], axis=1).copy()
        df['Outcome']= f.split(".")[0].split('#')[1]
        xQTL=f.split(".")[0].split('#')[2]
        df['xQTL_type'] =xQTL
        df_drop = df.dropna(subset=['Causal Estimate','Sd','P-value']).copy()
        df_drop=df_drop.sort_values(by=["MR Analysis"]).drop_duplicates(subset=['Exposure']).reset_index()
        p_adjusted = multipletests(df_drop["P-value"].tolist(), alpha=0.05, method='fdr_bh', is_sorted=False, returnsorted=False)[1]
        df_drop['FDR']=p_adjusted
        df_drop['method']="MRpresso"
        df_MP = pd.concat([df_drop, df_MP])

In [8]:
df_Maxlik.to_csv("./%s/%s_MaxLik_%s.tsv" % (DIR_mr_processed,file_type,update_date),index=False,sep='\t')
df_Maxlik.head()

,Exposure,Outcome,SNPs,method,Estimate,StdError,CILower,CIUpper,Pvalue,Heter.Stat[1],Heter.p,FDR,method_type,xQTL_type
0,CTSO,AD_Jansen_2019N,11,MaxLik,-0.005085,0.002658,-0.010295,0.000126,0.055785,9.093810,0.523224,0.417923,maxlik,Flead_map_eQTL_brain_rosmap_fdr
1,THAP9-AS1,AD_Jansen_2019N,4,MaxLik,-0.002743,0.002410,-0.007467,0.001980,0.254999,1.911446,0.590988,0.689707,maxlik,Flead_map_eQTL_brain_rosmap_fdr
2,TET2,AD_Jansen_2019N,6,MaxLik,0.001995,0.004340,-0.006512,0.010501,0.645814,7.087317,0.214227,0.887790,maxlik,Flead_map_eQTL_brain_rosmap_fdr
3,REST,AD_Jansen_2019N,3,MaxLik,-0.003133,0.005653,-0.014213,0.007946,0.579379,0.064183,0.968418,0.853859,maxlik,Flead_map_eQTL_brain_rosmap_fdr
4,POLN,AD_Jansen_2019N,3,MaxLik,0.004713,0.007032,-0.009069,0.018496,0.502693,2.709089,0.258065,0.835806,maxlik,Flead_map_eQTL_brain_rosmap_fdr


In [9]:
df_MP.rename(columns={'Sd':'StdError','P-value':'Pvalue','Causal Estimate':'Estimate',
                            "MR Analysis":"method_type"},inplace=True)
df_MP.to_csv("./%s/%s_MRpresso_%s.tsv" % (DIR_mr_processed,file_type,update_date),index=False,sep='\t')
df_MP.head()

,index,Exposure,method_type,Estimate,StdError,Pvalue,Outcome,xQTL_type,FDR,method
0,45,ZNF813,Outlier-corrected,-0.043925,0.025868,0.231597,AD_Kunkle_Stage1_2019N,Flead_map_eQTL_brain_mayocer_fdr,0.517233,MRpresso
1,83,C1orf112,Outlier-corrected,0.011748,0.029241,0.714761,AD_Kunkle_Stage1_2019N,Flead_map_eQTL_brain_mayocer_fdr,0.845100,MRpresso
2,367,PCDHB14,Outlier-corrected,-0.043900,0.021961,0.295295,AD_Kunkle_Stage1_2019N,Flead_map_eQTL_brain_mayocer_fdr,0.568641,MRpresso
3,0,GUCA1B,Raw,-0.014213,0.014127,0.343818,AD_Kunkle_Stage1_2019N,Flead_map_eQTL_brain_mayocer_fdr,0.597056,MRpresso
4,258,PPAP2B,Raw,-0.018472,0.027712,0.541525,AD_Kunkle_Stage1_2019N,Flead_map_eQTL_brain_mayocer_fdr,0.751855,MRpresso


In [10]:
df_WM.to_csv("./%s/%s_WM_%s.tsv" % (DIR_mr_processed,file_type,update_date),index=False,sep='\t')
df_WM.head()

,Exposure,Outcome,SNPs,method,Estimate,StdError,CILower,CIUpper,Pvalue,FDR,method_type,iterations,xQTL_type
0,GNRHR,AD_Kunkle_Stage1_2019N,15,WeightedMedian,-0.070216,0.026397,-0.121952,-0.018479,0.007814,0.356919,Weighted_Median,10000,Flead_map_eQTL_brain_mayometa_fdr
1,SCRG1,AD_Kunkle_Stage1_2019N,44,WeightedMedian,-0.011429,0.018172,-0.047045,0.024187,0.529383,0.934970,Weighted_Median,10000,Flead_map_eQTL_brain_mayometa_fdr
2,ZCRB1,AD_Kunkle_Stage1_2019N,19,WeightedMedian,-0.001228,0.032358,-0.064649,0.062193,0.969729,0.993078,Weighted_Median,10000,Flead_map_eQTL_brain_mayometa_fdr
3,RP6-91H8.5,AD_Kunkle_Stage1_2019N,14,WeightedMedian,0.042106,0.031094,-0.018837,0.103048,0.175686,0.821960,Weighted_Median,10000,Flead_map_eQTL_brain_mayometa_fdr
4,RP11-806K15.1,AD_Kunkle_Stage1_2019N,12,WeightedMedian,0.020580,0.034908,-0.047839,0.088999,0.555496,0.946045,Weighted_Median,10000,Flead_map_eQTL_brain_mayometa_fdr


In [11]:
#Combined all MR reuslts by different methods.
df_all = pd.concat([df_Wald_IVW,df_Maxlik,df_WM,df_MP])
df_all.to_csv("./%s/%s_MRall_%s_#%s.tsv" % (DIR_mr_processed,file_type,update_date,df_all.shape[0]),index=False,sep='\t')
df_all.shape

(1096930, 16)

In [12]:
#Step2, re-organized files names of GWAS and xQTL data.
reps = {
    'Flead-eQTL_metabrain_BasalGanglia':'eQTL_Metabrain_BasalGanglia',
    'Flead-eQTL_metabrain_Cerebellum':'eQTL_Metabrain_Cerebellum',
    'Flead-eQTL_metabrain_CortexAFA':'eQTL_Metabrain_CortexAFA',
    'Flead-eQTL_metabrain_Cortex':'eQTL_Metabrain_Cortex',
    'Flead-eQTL_metabrain_Hippocampus':'eQTL_Metabrain_Hippocampus',
    'Flead-eQTL_metabrain_SpinalCord':'eQTL_Metabrain_SpinalCord',
    'Flead_map_eQTL_brain_mayocer_fdr':'eQTL_Mayo_Cerebellum',
    'Flead_map_eQTL_brain_mayometa_fdr':'eQTL_Mayo_MetaAnalysis',
    'Flead_map_eQTL_brain_mayotcx_fdr':'eQTL_Mayo_Cortex',
    'Flead_map_eQTL_brain_rosmap_fdr':'eQTL_ROSMAP_Cortex',
    'Flead_map_pQTL_brain_banner_fdr':'pQTL_Banner_Cortex',
    'Flead_map_pQTL_brain_rosmapall_fdr':'pQTL_ROSMAP_Cortex.All',
    'Flead_map_pQTL_brain_rosmapcontrol_fdr':'pQTL_ROSMAP_Cortex.Ctl',
    'AD_Bellenguez_2022N':'AD_Bellenguez_2022',
    'AD_NMAFallNoUKB23_b37_2021N':'AD.NoUKB23_Wightman_2021',
    'AD_Kunkle_AFA_M2_b37_2020N':'AD.AAsAPOE_Kunkle_2020',
    'AD_Kunkle_AFA_M1_b37_2020N':'AD.AAs_Kunkle_2020',
    'AD_Jansen_2019N':'AD_Jansen_2019',
    'AD_GCST90012878_b37_2021N':'AD.FamilyHtx_Schwartzentruber_2021',
    'AD_GCST90012877_b37_2021N':'AD_Schwartzentruber_2021',
    'AD_Kunkle_Stage2_2019N':'AD.Stage2_Kunkle_2019',
    'AD_Kunkle_Stage1_2019N':'AD.Stage1_Kunkle_2019',
    'Inverse variance weighted':'IVW',
    'Wald ratio':'WaldRatio'
    }

def replace_all(text, reps_dic):
    for i, j in reps_dic.items():
        text = text.replace(i, j)
    return text

listname_mr = os.listdir("./%s" % DIR_mr_processed)
for filename in listname_mr:
    if ".tsv" in filename:
        with open("./%s/%s" % (DIR_mr_processed,filename),'r') as f:
            text=f.read()
            text=replace_all(text,reps)
        
        with open("./%s/processed-%s" % (DIR_mr_processed,filename),'w') as w:
            w.write(text)

In [13]:
#Combine MR results from Kunkle.Stage1 and Stage2 to Stage1&2
listname_mr = os.listdir("./%s" % DIR_mr_processed)
for filename in listname_mr:
    if "processed-" in filename:
        df_processed=pd.read_csv(f"./{DIR_mr_processed}/{filename}", sep="\t", low_memory=False)
        
        df_KunkleStage = df_processed[
            ((df_processed['Outcome'] == "AD.Stage1_Kunkle_2019") | (df_processed['Outcome'] == "AD.Stage2_Kunkle_2019")) & 
            (df_processed['xQTL_type'] != "eQTL_Metabrain_CortexAFA") 
        ]
        df_KunkleStage = df_KunkleStage.sort_values(by=['Pvalue'], ascending=True).drop_duplicates(subset= ['Exposure', 'xQTL_type'])
        df_KunkleStage.loc[:, 'Outcome'] = df_KunkleStage['Outcome'].str.replace("Stage1", "Stage1&2")
        df_KunkleStage.loc[:, 'Outcome'] = df_KunkleStage['Outcome'].str.replace("Stage2", "Stage1&2")        
        df_others=df_processed[(df_processed['Outcome'] != "AD.Stage1_Kunkle_2019") & 
                               (df_processed['Outcome'] != "AD.Stage2_Kunkle_2019") & 
                               (df_processed['xQTL_type'] != "eQTL_Metabrain_CortexAFA")]
        df_processed_update = pd.concat([df_KunkleStage,df_others]).reset_index(drop=True)
        name = filename.split(".")[0]
        print(f"{name}#{df_processed_update.shape[0]}")
        df_processed_update.to_csv(f"./{DIR_mr_processed}/update_{filename}", sep='\t',index=False)

processed-AD#LD02_IVW3_Jul17#192106
processed-AD#LD02_snp2_Jul17#361082
processed-AD#LD02_WM_Jul17#192098
processed-AD#LD02_IVW_Jul17#552983
processed-AD#LD02_MRall_Jul17_#1096930#1008269
processed-AD#LD02_MaxLik_Jul17#192098
processed-AD#LD02_MRpresso_Jul17#135732


### Count significant MR results across all xQTL* AD_GWAS pairs.
sig#: n-m, where n represents the number of significant MR results with β-coefficient > 0 at FDR < 0.05 in EA (European American) and P < 0.05 in AA (African American); m represents the number of significant MR results with β-coefficient < 0 at FDR < 0.05 in EA or P < 0.05 in AA. 

In [14]:
def unique(list1): 
    x = np.array(list1) 
    return set(np.unique(x))
def count_posi(k,test_list):
    count = len([i for i in test_list if i > k])
    return count
def count_neg(k,test_list):
    count = len([i for i in test_list if i < k])
    return count
def count_var(k,test_list):
    count = len([i for i in test_list if i == k])
    return count

def MR_count(df,MR_type,sig_type,cutoff,sample_type,update_time,PATH_out):
    #generate a MR score and save in PATH_out dir.
    df_count = df.groupby(['Exposure']).count()
    df_count=df_count.sort_values(['method'], ascending=False).rename(columns={'method':'total#'})
    print(f"How many drug targets for {sample_type} at {sig_type}< 0.05:#{df_count.shape[0]}")
    df_count_sub=df_count[["total#"]].copy()
    df_sig=df[df[sig_type]<0.05]
    sig_gene_list=unique(df_sig["Exposure"].tolist())
    for gene in df_count_sub.index:
        gene_sub_all=df[df['Exposure']==gene]
        EST_list_all=gene_sub_all['Estimate'].astype(float).tolist()
        posi_count_all=count_posi(0,EST_list_all)
        df_count_sub.loc[gene,"posi#"]=posi_count_all
        neg_count_all=count_neg(0,EST_list_all)
        df_count_sub.loc[gene,"neg#"]=neg_count_all
        if gene in sig_gene_list:
            gene_sub=df_sig[df_sig['Exposure']==gene]
            EST_list=gene_sub['Estimate'].astype(float).tolist()
            n=count_posi(0,EST_list)
            df_count_sub.loc[gene,"n.#sig_posi"]=int(n)
            m=count_neg(0,EST_list)
            df_count_sub.loc[gene,"m.#sig_neg"]=int(m)
            df_count_sub.loc[gene,"sig#"]=n-m
            df_count_sub.loc[gene,"abs_sig#"]=n+m

    df_count_sub=df_count_sub.reset_index().rename(columns={"Exposure":"symbol"})
    print(df_count_sub.head())
    df_count_sub.to_csv("%s/%s_%s%s_%s_score_%s_sig.tsv" % (PATH_out,MR_type,sig_type,cutoff,sample_type,update_time), sep="\t", index=False)
    return df_count_sub

In [15]:
#Filter the MR results for Drug Targets.
df_MR_IVW = pd.read_csv(f"./{DIR_mr_processed}/update_processed-{file_type}_IVW_{update_date}.tsv",sep="\t",low_memory=False)
df_target = pd.read_csv(f"./target_#1415.tsv",sep="\t")
df_target.rename(columns={'symbol': "Exposure"},inplace = True)
df_DT = pd.merge(df_MR_IVW,df_target,how = "inner",on =["Exposure"])
print(f"{df_DT.shape[0]}")
print("=======end======")
df_DT.to_csv(f"./{DIR_mr}/DT-{file_type}_IVW_{update_date}_#{df_DT.shape[0]}.tsv", sep='\t',index=False)

38627
=======end======


In [16]:
#Calculating the number of significant for Drug Targets
#Drug targets are defined based on a binding affinity less than 1000nM between the protein and ligand.

#EA-specific
df_EUR=df_DT[(df_DT['Outcome'] != 'AD.AAsAPOE_Kunkle_2020') & (df_DT['Outcome'] != "AD.AAs_Kunkle_2020")].copy()
#print(f"The number of MR results for EA: #{df_EUR.shape[0]}")
df_EUR.to_csv(f"./{DIR_mr_processed}/update_processedEUR-{file_type}_IVW_{update_date}.tsv", sep='\t',index=False)
EUR_q=MR_count(df_EUR,file_type,'FDR',0.05,'DT_EUR',update_date,DIR_alzMR)
EUR_p=MR_count(df_EUR,file_type,'Pvalue',0.05,'DT_EUR',update_date,DIR_alzMR)
print("===============")
#AA-specific
df_AA=df_DT[(df_DT['Outcome'] == 'AD.AAsAPOE_Kunkle_2020') | 
                               (df_DT['Outcome'] == "AD.AAs_Kunkle_2020") ].copy()
#print(f"The number of MR results for AA: #{df_AA.shape[0]}")
df_AA.to_csv(f"./{DIR_mr_processed}/update_processedAA-{file_type}_IVW_{update_date}.tsv", sep='\t',index=False)
AA_q=MR_count(df_AA,file_type,'FDR',0.05,'DT_AA',update_date,DIR_alzMR)
AA_p=MR_count(df_AA,file_type,'Pvalue',0.05,'DT_AA',update_date,DIR_alzMR)

How many drug targets for DT_EUR at FDR< 0.05:#1229
  symbol  total#  posi#  neg#  n.#sig_posi  m.#sig_neg  sig#  abs_sig#
0  EPHX2      72   70.0   2.0         22.0         0.0  22.0      22.0
1   PEPD      72    8.0  64.0          0.0         2.0  -2.0       2.0
2    GAA      66   33.0  33.0          2.0         0.0   2.0       2.0
3   NQO2      66   28.0  38.0          NaN         NaN   NaN       NaN
4  SIRT5      66   20.0  46.0          NaN         NaN   NaN       NaN
How many drug targets for DT_EUR at Pvalue< 0.05:#1229
  symbol  total#  posi#  neg#  n.#sig_posi  m.#sig_neg  sig#  abs_sig#
0  EPHX2      72   70.0   2.0         40.0         1.0  39.0      41.0
1   PEPD      72    8.0  64.0          0.0        14.0 -14.0      14.0
2    GAA      66   33.0  33.0          6.0         4.0   2.0      10.0
3   NQO2      66   28.0  38.0          2.0         0.0   2.0       2.0
4  SIRT5      66   20.0  46.0          0.0         1.0  -1.0       1.0
How many drug targets for DT_AA at FDR< 0

In [17]:
def preprocess_df(df, suffix):
    df = df.add_suffix(f'_{suffix}')
    columns_to_keep = [f"symbol_{suffix}", f"n.#sig_posi_{suffix}", f"m.#sig_neg_{suffix}", f"sig#_{suffix}", f"abs_sig#_{suffix}"]
    df = df[columns_to_keep].rename(columns={f"symbol_{suffix}":"symbol"})
    return df

df_AAp = preprocess_df(AA_p, 'AA.P')
df_EAp = preprocess_df(EUR_p, 'EA.P')
df_AAp.fillna(0, inplace=True)
df_EAp.fillna(0, inplace=True)

df_AA_EA = pd.merge(df_AAp, df_EAp, how="outer", on="symbol")
df_AA_EA['AA-EA_ratio'] = (df_AA_EA['abs_sig#_AA.P'] - df_AA_EA['abs_sig#_EA.P'])
df_AA_EA = df_AA_EA.sort_values(by='AA-EA_ratio', ascending=False)
df_AA_EA_ratio = df_AA_EA[df_AA_EA['AA-EA_ratio'] >= 2][['symbol', 'abs_sig#_AA.P', 'AA-EA_ratio']]
#df_AA_EA_ratio.to_csv(f"./{DIR_alzMR}/alzScores_DT_EA_AA_ratio.tsv", index=False, sep="\t")
df_AA_EA_ratio.head()


,symbol,abs_sig#_AA.P,AA-EA_ratio
331,SETD7,5.0,5.0
274,ABL1,4.0,4.0
44,ALDH2,4.0,4.0
565,SGK1,4.0,4.0
104,TRPV3,6.0,4.0


## alzMR-Scores

In [18]:
#Function for alzMR-Scores
def calculate_alzMRscores(data):
    # Create score for each gene
    score = data.groupby('Exposure').agg(
        K=('Z', 'size'),
        SG=('Z', 'sum'),
        meanBeta=('Estimate', 'mean'),
        meanPosBeta=('Estimate', lambda x: x[x > 0].mean() if not x[x > 0].empty else 0),
        meanNegBeta=('Estimate', lambda x: x[x < 0].mean() if not x[x < 0].empty else 0),
        nPosBeta=('Estimate', lambda x: (x > 0).sum()),
        nNegBeta=('Estimate', lambda x: (x < 0).sum())
    ).reset_index()

    score['SGK'] = score['SG'] / score['K']

    # Rankings
    score = score.reindex(score.SG.abs().sort_values(ascending=False).index)
    score['rankSumZscores'] = np.arange(1, len(score) + 1)
    score = score.reindex(score.SGK.abs().sort_values(ascending=False).index)
    score['rankMeanZscores'] = np.arange(1, len(score) + 1)
    score = score.sort_values(by='rankSumZscores')

    score = score[['Exposure', 'SG', 'SGK', 'rankSumZscores', 'rankMeanZscores']]
    score.columns = ['symbol', 'alzMRScore', 'Nor-alzMRScore', 'rankSumZscores', 
                     'rankMeanZscores']
    score.reset_index(drop=True,inplace=True)
    return score


In [19]:
# Calculate Z-statistics from beta/se
df_EUR['Z'] = df_EUR['Estimate'] / df_EUR['StdError']
MRscore = calculate_alzMRscores(df_EUR)
MRscore=pd.merge(MRscore,EUR_q, how="left",on="symbol")

MRscore.to_csv(f"./{DIR_alzMR}/alzScores_DT_EUR_Zstatistics.tsv", index=False, sep="\t")
MRscore.head()

,symbol,alzMRScore,Nor-alzMRScore,rankSumZscores,rankMeanZscores,total#,posi#,neg#,n.#sig_posi,m.#sig_neg,sig#,abs_sig#
0,ACE,-199.231645,-4.150659,1,2,48,0.0,48.0,0.0,32.0,-32.0,32.0
1,EPHX2,195.698354,2.718033,2,10,72,70.0,2.0,22.0,0.0,22.0,22.0
2,LACTB,165.475084,3.064353,3,9,54,54.0,0.0,21.0,0.0,21.0,21.0
3,ADAM10,133.563373,3.180080,4,7,42,42.0,0.0,17.0,0.0,17.0,17.0
4,ERCC2,126.195368,3.505427,5,5,36,33.0,3.0,20.0,0.0,20.0,20.0


In [20]:
# Calculate Z-statistics from beta/se
df_AA['Z'] = df_AA['Estimate'] / df_AA['StdError']
MRscore_AA = calculate_alzMRscores(df_AA)
MRscore_AA=pd.merge(MRscore_AA,df_AA_EA_ratio, how="inner",on="symbol")
MRscore_AA.to_csv(f"./{DIR_alzMR}/alzScores_DT_AA_Zstatistics.tsv", index=False, sep="\t")
MRscore_AA.head()

,symbol,alzMRScore,Nor-alzMRScore,rankSumZscores,rankMeanZscores,abs_sig#_AA.P,AA-EA_ratio
0,EBPL,-30.367974,-1.897998,1,12,6.0,2.0
1,BCR,24.016763,1.501048,4,41,4.0,2.0
2,DHFR,21.116581,1.173143,7,111,3.0,2.0
3,ALDH2,-20.740114,-1.296257,8,72,4.0,4.0
4,SLC7A11,20.707722,1.294233,9,73,3.0,2.0
